In [ ]:
import requests
import json
import pandas as pd
import ipywidgets as w
import ipython_blocking

In [ ]:
client_id = '07bd2676-f950-48c0-8b12-ebd5e8b1491d'
client_secret = 'b874aa02-4333-4a4a-be10-55b44fc94eb3'


In [ ]:
def auth():
    _auth_url = 'http://agrows-keycloak.labbs.com.br/auth/realms/agroWS/protocol/openid-connect/token'
    _auth_data = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    _auth_headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    _auth_request = requests.post(_auth_url, data=_auth_data, headers=_auth_headers)
    return json.loads(_auth_request.text)['access_token']

In [ ]:
def load_data(_owner, _thing, _node, _token, _start_date, _end_date):
    _data_api_url = f'https://agrows-data-api.labbs.com.br/v1/owner/{_owner}/thing/{_thing}/node/{_node}'
    _data_api_params = {
        'startDateTime': _start_date,
        'endDateTime': _end_date
    }
    _data_api_headers = {
        'content-type': 'application/json',
        'Authorization' : f'Bearer {_token}'
    }
    _data_api_request = requests.get(_data_api_url, params=_data_api_params, headers=_data_api_headers)
    _data = json.loads(_data_api_request.text)['data']
    # data_source = json.dumps(data, indent=4, sort_keys=True)
    return _data

In [ ]:
def load_mongo_data(_repoList):
    _data_api_url = f'http://localhost:3005/'
    _data_api_params = {
        'repoList': _repoList
    }
    _data_api_headers = {
        'content-type': 'application/json'
    }
    _data_api_request = requests.get(_data_api_url, params=_data_api_params, headers=_data_api_headers)
    _data = json.loads(_data_api_request.text)['data']
    print(_data)
    # data_source = json.dumps(data, indent=4, sort_keys=True)
    return _data

In [ ]:
def btn_click(evt):
    _start_date = str(start_date_widget.value) + 'T00:00:00-03:00'
    _end_date = str(end_date_widget.value) + 'T23:59:59-03:00'
    _owner = owner_widget.value
    _thing = thing_widget.value
    _node = node_widget.value
    _data = load_data(owner, thing, node, auth(), start_date, end_date)
    _data_source = [{'dateTime': x['dateTime'], 'owner': x['owner'],'thing': x['thing'],'node': x['node'], **x.get('attributes')} for x in data]
    _df = pd.DataFrame.from_dict(_data_source)
    display(_df)
    return _df

In [ ]:
owner_widget = w.Dropdown(
    options=['gitfeed'],
    value='gitfeed',
    description='Owner:',
    disabled=False,
)
thing_widget = w.Dropdown(
    options=['github', 'gitlab'],
    value='github',
    description='Provider:',
    disabled=False,
)
node_widget = w.Dropdown(
    options=['pulls', 'issues', 'commits', 'userStatsV1'],
    value='pulls',
    description='Node:',
    disabled=False,
)
start_date_widget = w.DatePicker(
    description='Start Date',
    disabled=False
)
end_date_widget = w.DatePicker(
    description='End Date',
    disabled=False
)

search_btn = w.Button(
    description='Pesquisar',
    disabled=False,
    button_style='success',
    tooltip='Pesquisar',
    icon='check'
)
display(start_date_widget, end_date_widget, search_btn)
# search_btn.on_click(btn_click)

In [ ]:
%blockrun search_btn

start_date = str(start_date_widget.value) + 'T00:00:00-03:00'
end_date = str(end_date_widget.value) + 'T23:59:59-03:00'
owner = owner_widget.value
thing = thing_widget.value
node = node_widget.value
data = load_data(owner, thing, node, auth(), start_date, end_date)
data_source = [{'dateTime': x['dateTime'], 'owner': x['owner'],'thing': x['thing'],'node': x['node'], **x.get('attributes')} for x in data]
df = pd.DataFrame.from_dict(data_source)